In [2]:
import datasets

book_builder = datasets.load_dataset_builder("wikitext")
print(book_builder.info.description)
print(book_builder.info.splits)
print(book_builder.info.features)

/Users/vik/.virtualenvs/nnets/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Config name is missing.
Please pick one among the available configs: ['wikitext-103-v1', 'wikitext-2-v1', 'wikitext-103-raw-v1', 'wikitext-2-raw-v1']
Example of usage:
	`load_dataset('wikitext', 'wikitext-103-v1')`

In [70]:
wrapper = DatasetWrapper()

wiki = datasets.load_dataset("wikitext", "wikitext-103-v1", split="train[:1%]")
wiki = wiki.map(combine_wikitext, batched=True, remove_columns=wiki.column_names)
tokenizer = wrapper.get_tokenizer(wiki[wrapper.data_key])
tokenized = wiki.map(lambda examples: get_tokens_and_ids(examples, tokenizer, wrapper.data_key, wrapper.ids_key, wrapper.tokens_key), batched=True)

Found cached dataset wikitext (/Users/vik/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Loading cached processed dataset at /Users/vik/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-15a8e8a993ee0984.arrow
Loading cached processed dataset at /Users/vik/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-3af97c1d5bae50bf.arrow


In [85]:
chunked

Dataset({
    features: ['chunked_tokens', 'chunked_input_ids'],
    num_rows: 3111
})

In [82]:
wrapper = DatasetWrapper(512)
data = wrapper.process_dataset()

Found cached dataset wikitext (/Users/vik/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
100%|██████████| 1/1 [00:01<00:00,  1.13s/ba]


In [86]:
data[0]

{'chunked_tokens': ['▁S',
  'en',
  'j',
  '▁no',
  '▁V',
  'al',
  'ky',
  'ri',
  'a',
  '▁3',
  '▁:',
  '▁',
  '<unk>',
  '▁Ch',
  'ron',
  'ic',
  'les',
  '▁(',
  '▁Japan',
  'es',
  'e',
  '▁:',
  '▁3',
  '▁,',
  '▁lit',
  '▁.',
  '▁V',
  'al',
  'ky',
  'ri',
  'a',
  '▁of',
  '▁the',
  '▁Battle',
  'f',
  'iel',
  'd',
  '▁3',
  '▁)',
  '▁,',
  '▁commonly',
  '▁referred',
  '▁to',
  '▁as',
  '▁V',
  'al',
  'ky',
  'ri',
  'a',
  '▁Ch',
  'ron',
  'ic',
  'les',
  '▁III',
  '▁outside',
  '▁Japan',
  '▁,',
  '▁is',
  '▁a',
  '▁t',
  'act',
  'ical',
  '▁ro',
  'le',
  '▁@-@',
  '▁play',
  'ing',
  '▁video',
  '▁game',
  '▁devel',
  'oped',
  '▁by',
  '▁Se',
  'ga',
  '▁and',
  '▁M',
  'edia',
  '.',
  'V',
  'ision',
  '▁for',
  '▁the',
  '▁Pl',
  'ay',
  'S',
  't',
  'ation',
  '▁Port',
  'able',
  '▁.',
  '▁Releas',
  'ed',
  '▁in',
  '▁January',
  '▁2011',
  '▁in',
  '▁Japan',
  '▁,',
  '▁it',
  '▁is',
  '▁the',
  '▁third',
  '▁game',
  '▁in',
  '▁the',
  '▁V',
  'al',
  'ky

In [81]:
from tokenizers import SentencePieceBPETokenizer
from transformers import PreTrainedTokenizerFast
import os
import re

def chunk_tokens_and_ids(examples, chunk_size, ids_key, tokens_key):
    chunked_ids = []
    chunked_tokens = []
    for id, token in zip(examples[ids_key], examples[tokens_key]):
        chunked_ids += [id[i:i + chunk_size] for i in range(0, len(id), chunk_size)]
        chunked_tokens += [token[i:i + chunk_size] for i in range(0, len(token), chunk_size)]
    return {f"chunked_{tokens_key}": chunked_tokens, f"chunked_{ids_key}": chunked_ids}

def get_tokens_and_ids(examples, tokenizer, text_key, ids_key, tokens_key):
    tokens = []
    ids = []
    for example in examples[text_key]:
        token = tokenizer.tokenize(example, padding=False)
        id = tokenizer.convert_tokens_to_ids(token)
        tokens.append(token)
        ids.append(id)
    return {tokens_key: tokens, ids_key: ids}

def calc_token_ratio(examples, ids_key, data_key):
    ratios = []
    for id, data in zip(examples[ids_key], examples[data_key]):
        ratios.append(len(id) / len(data))
    return {"token_ratio": ratios}

class DatasetWrapper:
    dataset_name = "wikitext"
    data_config = "wikitext-103-v1"
    download_split = "train"
    download_split_pct = "1%"
    data_key = "text"
    ids_key = "input_ids"
    tokens_key = "tokens"
    max_token_ratio = .33

    def __init__(self, model_max_length=512):
        self.tokenizer_filename = f"{self.dataset_name}_{self.download_split_pct}_tokenizer"
        self.model_max_length = model_max_length

    def dataset_info(self):
        data = datasets.load_dataset_builder(self.dataset_name)
        print(data.info.description)
        print(data.info.splits)
        print(data.info.features)

    def process_dataset(self):
        data = self.load_dataset()
        if self.combine_func is not None:
            data = data.map(lambda x: self.combine_func(x, text_key=self.data_key), batched=True, remove_columns=data.column_names)

        tokenizer = self.get_tokenizer(data[self.data_key])
        tokenized = self.tokenize_dataset(data, tokenizer)
        tokenized = self.filter_tokenized_text(tokenized)
        data_chunks = self.chunk_tokens(tokenized)
        return data_chunks

    def combine_func(self, examples, text_key="text"):
        entries = []
        entry = ""
        for sentence in examples[text_key]:
            if re.match("^ \= \w", sentence):
                if entry:
                    entries.append(entry)
                entry = ""
            else:
                entry += sentence
        entries.append(entry)
        return {text_key: entries}

    def load_dataset(self):
        split_str = self.download_split
        if self.download_split_pct:
            split_str = f"{self.download_split}[:{self.download_split_pct}]"
        if not self.data_config:
            data = datasets.load_dataset(self.dataset_name, split=split_str)
        else:
            data = datasets.load_dataset(self.dataset_name, self.data_config, split=split_str)
        return data

    def tokenize_dataset(self, data, tokenizer):
        data = data.map(lambda examples: get_tokens_and_ids(examples, tokenizer, self.data_key, self.ids_key, self.tokens_key), batched=True)
        return data

    def filter_tokenized_text(self, data):
        data = data.map(lambda x: calc_token_ratio(x, self.ids_key, self.data_key), batched=True)
        data = data.filter(lambda x: x["token_ratio"] < self.max_token_ratio)
        return data

    def chunk_tokens(self, data):
        data_chunks = data.map(lambda examples: chunk_tokens_and_ids(examples, self.model_max_length, self.ids_key, self.tokens_key), batched=True, remove_columns=data.column_names)
        return data_chunks

    def get_tokenizer(self, data=None):
        if os.path.exists(self.tokenizer_filename):
            tokenizer = self.load_tokenizer(self.tokenizer_filename)
        else:
            tokenizer = self.train_tokenizer(data, self.tokenizer_filename)
        return tokenizer

    def train_tokenizer(self, text, save_file, vocab_size=5000, min_frequency=2):
        special_tokens = ["<s>", "<pad>", "</s>", "<unk>"]
        sptokenizer = SentencePieceBPETokenizer()
        sptokenizer.train_from_iterator(
            text,
            vocab_size=vocab_size,
            min_frequency=min_frequency,
            show_progress=True,
            special_tokens=special_tokens
        )

        tokenizer = PreTrainedTokenizerFast(tokenizer_object=sptokenizer, special_tokens=special_tokens)
        for attr, token in zip(["bos_token", "pad_token", "eos_token", "unk_token"], special_tokens):
            setattr(tokenizer, attr, token)
            setattr(tokenizer, f"{attr}_id", sptokenizer.token_to_id(token))
        tokenizer.save_pretrained(save_file)
        return tokenizer

    def load_tokenizer(self, tokenizer_file):
        tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_file)
        return tokenizer